In [23]:
# Patch Experta for Python 3.10+ Compatibility
import collections
import collections.abc
collections.Mapping = collections.abc.Mapping

In [ ]:
# Install dependencies (run only once in Colab)
!pip install experta spacy nltk
!python -m spacy download en_core_web_sm

In [25]:
import pandas as pd
import numpy as np
import ast
import difflib
import string
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from experta import Fact, Rule, DefFacts, KnowledgeEngine, MATCH


In [26]:
# NLTK Data Initialization
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# ---------- Load & Clean Data ----------


In [27]:
file_path = "/content/Career Guidance Expert System.csv"
data = pd.read_csv(file_path)

In [28]:
# Replace empty lists with NaN for handling
data.replace({"[]": np.nan}, inplace=True)

In [29]:
# Convert skill strings to actual lists
def to_list(entry):
    try:
        return [item.strip().lower() for item in ast.literal_eval(entry)]
    except Exception:
        return []

data["hard_skill"] = data["hard_skill"].apply(to_list)
data["soft_skill"] = data["soft_skill"].apply(to_list)



In [30]:
# Fill missing skills with mode values for each field
def impute_missing_skills(df):
    hard_mode = df.groupby("candidate_field")["hard_skill"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else [])
    soft_mode = df.groupby("candidate_field")["soft_skill"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else [])

    def fill(row):
        if not row["hard_skill"]:
            row["hard_skill"] = hard_mode.get(row["candidate_field"], [])
        if not row["soft_skill"]:
            row["soft_skill"] = soft_mode.get(row["candidate_field"], [])
        return row

    return df.apply(fill, axis=1)

data = impute_missing_skills(data)

# ---------- Knowledge Base Construction ----------


In [31]:
# Build a skill map from dataset
career_map = {
    field: {
        "hard": set(sum(group["hard_skill"].tolist(), [])),
        "soft": set(sum(group["soft_skill"].tolist(), []))
    }
    for field, group in data.groupby("candidate_field")
}

In [32]:
# Get all known skills
known_skills = sorted(set().union(*(entry["hard"] | entry["soft"] for entry in career_map.values())))

# ---------- NLP Initialization ----------

In [33]:
# Ensure NLTK data is downloaded before use
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [36]:
# Define the extract_terms function
def extract_terms(text):
    """Tokenizes, removes stop words, and lemmatizes the input text."""
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if token.text not in string.punctuation]
    tokens = [token for token in tokens if token not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

# ---------- Expert System using Experta ----------


In [37]:
class Profile(Fact):
    """User-provided skill description"""
    pass

class Advisor(KnowledgeEngine):

    @DefFacts()
    def _init_facts(self):
        yield Fact(intent="analyze")

    @Rule(Fact(intent="analyze"), Profile(description=MATCH.description))
    def suggest_career(self, description):
        tokens = extract_terms(description)
        matched = []

        for token in tokens:
            best_match, best_score = None, 0
            for reference in known_skills:
                score = difflib.SequenceMatcher(None, token, reference).ratio()
                if score > best_score:
                    best_match, best_score = reference, score
            if best_score >= 0.6:
                matched.append(best_match)

        ranking = {
            field: sum(skill in skills["hard"] for skill in matched) +
                   sum(skill in skills["soft"] for skill in matched)
            for field, skills in career_map.items()
        }

        if any(ranking.values()):
            recommendation = max(ranking, key=ranking.get)
            print(f"\n✅ Recommended Career Field: **{recommendation.title()}**")
        else:
            print("\n⚠️ No suitable match found. Please revise your input.")


# ---------- Run System ----------

In [39]:
# Get user input and run the expert system
user_input = input("🧠 Describe your experience, skills, or projects:\n")

if user_input.strip():
    expert = Advisor()
    expert.reset()
    expert.declare(Profile(description=user_input))
    expert.run()
else:
    print("⚠️ Empty input. Please try again.")

🧠 Describe your experience, skills, or projects:
 "I have experience in nursing, registration,  and service. I am also good at written communication."

✅ Recommended Career Field: **Healthcare & Medical**
